# Sean

In [1]:
#Import the relevant Python modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

In [2]:
#Import the relevant data
Salmon = pd.read_excel("Data/ZooPlanktonPerryData.xlsx", 5)
oldSSTs = pd.read_csv("Data/SSTs")
Salinity = pd.read_csv("Data/Entrance_Island_-_Average_Monthly_Sea_Surface_Salinities_1936-2023.csv")
Catches = pd.read_excel("Data/Copy of recreational reported fishery catch, 1953-2012.xlsx")
SeaLevel = pd.read_csv("Data/SeaLevel_Cherry_annualMean.txt", sep=";")
xl_file = pd.ExcelFile('Data/ZooPlanktonPerryData.xlsx')


In [3]:
Salmon = Salmon.rename(columns={"Ocean Entry Year":"Year"})

#For each year I want a list with the SSTs for the months in ascending order. 
SSTsList = []
for year in oldSSTs["Year"].unique():
    SSTsList.append([year] + oldSSTs[oldSSTs["Year"] == year]["SST"].tolist())

SSTs = pd.DataFrame(SSTsList, columns=["Year", "SST Jan.", "SST Feb.", "SST Mar.", "SST Apr.", "SST May", "SST Jun.", "SST Jul.", "SST Aug", "SST Sep.", "SST Oct.", "SST Nov.", "SST Dec."])

#Extract the zoo plankton anomalies from the excel sheet
Original_Salmon_Sheets = {sheet_name: xl_file.parse(sheet_name) 
          for sheet_name in xl_file.sheet_names}

anom = Original_Salmon_Sheets['3. Zooplankton anomalies']
anom = anom[6:]
zoos = anom[anom.columns[3:]]
zoos["Year"] = anom["Year"].astype(int)

#Grab the relevant years from Salinity
Salinity = Salinity[55:] 
Salinity = Salinity.rename(columns={"ENTRANCE ISLAND LIGHTSTATION: AVERAGE MONTHLY SEA SURFACE SALINITIES (PSU)":"Year"})
Salinity = Salinity['Year'].astype(int)

#Next is Catches
ModernCatches = Catches[Catches["YEAR"] >= 1990]
CohoCatches = ModernCatches[ModernCatches["SPECIES_DESC"] == "COHO SALMON"]
ChinookCatches = ModernCatches[ModernCatches["SPECIES_DESC"] == "CHINOOK SALMON"]
cohoAnnualCatchList = []
chinookAnnualCatchList = []
yearList = []
for year in CohoCatches["YEAR"].unique():
    cohoAnnualCatchList.append(CohoCatches[CohoCatches["YEAR"] == year]["PIECES"].sum())
    chinookAnnualCatchList.append(ChinookCatches[ChinookCatches["YEAR"] == year]["PIECES"].sum())
    yearList.append(year)
Catches = pd.DataFrame({"Coho": cohoAnnualCatchList, "Chinook": chinookAnnualCatchList, "Year": yearList})
#Sealevel
SeaLevel = SeaLevel.drop(['Y', '000'], axis=1)
SeaLevel= SeaLevel.rename(columns={' 1985': "Year",'  6945': "Sea Level"}, errors="raise")
SeaLevel = SeaLevel[4:]


C:\Users\aycin\AppData\Local\Temp\ipykernel_7728\1967338990.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  zoos["Year"] = anom["Year"].astype(int)


In [4]:
#Combine the relevant data into a single data frame
newFrame1 = pd.merge(Salmon, SSTs, on="Year", how="outer")
newFrame2 = pd.merge(newFrame1, zoos, on="Year", how="outer")
newFrame3 = pd.merge(newFrame2, Salinity, on="Year", how="outer")
newFrame4 = pd.merge(newFrame3, Catches, on="Year", how="outer")
AllData = pd.merge(newFrame4, SeaLevel, on="Year", how="outer")
AllData[["Cowichan Chinook", "Harrison Chinook", "Puntledge Chinook"]] = AllData[["Cowichan Chinook", "Harrison Chinook", "Puntledge Chinook"]].fillna(0)
#Drop the years we have no viability data for
AllData = AllData.drop([28, 29, 30, 31, 32, 33])
AllDataNoNull = AllData.dropna()

## Obtained Data Frame

In [5]:
AllDataNoNull.head(3)

,Year,Cowichan Chinook,Harrison Chinook,Puntledge Chinook,Big Qualicum Coho,SST Jan.,SST Feb.,SST Mar.,SST Apr.,SST May,...,Natantia,NonCalCops,Ostracoda,PolychaetPelagic,Pteropods,Repantia,Siphonophorae,Coho,Chinook,Sea Level
6,1996,0.008811,0.010038,0.000000,0.014246,10.860983,12.025331,11.728321,11.119489,9.948300,...,0.102015,0.242330,0.297214,0.255998,0.019095,-0.604888,0.391865,217199.0,114942.0,6988.0
7,1997,0.008294,0.023246,0.004286,0.004293,11.910243,12.317101,12.166986,11.192175,10.135995,...,0.163304,0.248062,0.278687,-0.063418,-0.024925,0.211126,0.247333,213163.0,154268.0,7081.0
8,1998,0.010999,0.008297,0.017847,0.013055,12.177850,12.481543,12.182488,11.857747,10.362404,...,0.159563,0.238723,0.374939,0.089059,0.701012,0.770283,0.089499,16335.0,160780.0,7065.0


# Roy

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

start_date = 1996
end_date = 2017

#(1) read sea salinity data
Salinity_file = pd.read_excel('Data/Entrance_island_salinity.xlsx') 

#(2) read Pacific decadal oscillation(PDO) data
Pdo_file = pd.read_excel('Data/pdo.xlsx','pdo')  

#(3) read Fraser river flow data
Flow_rate_file = pd.read_excel('Data/Fraser_flow_rate.xlsx','Flow rate')  

#(4) read Killer Whale data
Whale_file = pd.read_excel('Data/killerwhales_1970to2020.xlsx','killerwhales_1970to2020') 

#(5) read harbour seal population data
Seal_file = pd.read_excel('Data/harbourseals_1970to2020.xlsx','harbourseals_1970to2020')  


In [7]:
#Sea salinity is provided monthly, we will calculate yearly average. Also, nonexistent data is marked by 999.99

df_salinity = Salinity_file.replace(999.99,np.nan)
df_salinity_yr = df_salinity['YEAR']
df_salinity_val = df_salinity.drop(['YEAR'],axis=1).mean(axis = 1)
df_sea_salinity = pd.DataFrame({'Year':df_salinity_yr,'Avg Sea Salinity':df_salinity_val})
df_sea_salinity = df_sea_salinity.loc[(df_sea_salinity['Year'] >= start_date) & (df_sea_salinity['Year'] <= end_date)]
df_sea_salinity.reset_index(drop = True, inplace = True)

In [8]:
df_pdo = Pdo_file.loc[(Pdo_file['year'] >= start_date) & (Pdo_file['year'] <= end_date)]
df_pdo_clean = df_pdo.rename(columns={'year':'Year'})
df_pdo_avg = df_pdo_clean.groupby(['Year']).mean()
df_pdo_avg = df_pdo_avg.drop(columns = ['date','month'])


df_flow_rate = Flow_rate_file[(Flow_rate_file['Year'] >= start_date) & (Flow_rate_file['Year'] <= end_date) &(Flow_rate_file['PARAM'] == 1)]
df_flow_rate = df_flow_rate.drop(columns = [' ID','PARAM','MM--DD','SYM','MM--DD.1','SYM.1'])
df_flow_rate['Avg flow rate'] = 0.5*(df_flow_rate['MAX']+df_flow_rate['MIN'])
df_flow_rate.rename(columns = {'MAX':'Max flow rate','MIN':'Min flow rate'},inplace = True)
df_flow_rate.reset_index(drop = True, inplace = True)

In [9]:
df_env_factors = pd.merge(df_sea_salinity,df_pdo_avg,on = 'Year',how = 'outer')
df_env_factors = pd.merge(df_env_factors, df_flow_rate[['Year','Max flow rate','Min flow rate','Avg flow rate']], on = 'Year', how = 'outer')
print(df_env_factors)

    Year  Avg Sea Salinity       pdo  Max flow rate  Min flow rate  \
0   1996         26.720833  0.683333         8100.0          675.0   
1   1997         25.359167  1.316667        11300.0          788.0   
2   1998         27.243333 -0.478333         6710.0          600.0   
3   1999         26.195833 -1.843333        11000.0          647.0   
4   2000         27.405833 -1.125833         8000.0          470.0   
5   2001         28.385000 -1.134167         7140.0          478.0   
6   2002         27.663333 -0.439167        10600.0          491.0   
7   2003         27.967273  0.381667         7300.0          472.0   
8   2004         27.438333 -0.224167         6650.0          560.0   
9   2005         27.265000 -0.188333         7460.0          805.0   
10  2006         28.180833 -0.350833         7190.0          510.0   
11  2007         26.918333 -0.702500        10800.0          560.0   
12  2008         27.835833 -1.663333        10200.0          648.0   
13  2009         27.

In [10]:
df_whale = Whale_file.loc[(Whale_file['Year'] >= start_date) & (Whale_file['Year'] <= end_date)].drop(columns = ['SRKW','srkwSource','nrkwSource'])
df_seal = Seal_file.loc[(Seal_file['year'] >= start_date) & (Seal_file['year'] <= end_date)].drop(columns = ['nonSogNt','bcNt']).rename(columns={'year':'Year'})


df_whale.reset_index(drop = True, inplace = True)
df_seal.reset_index(drop = True, inplace = True)

df_pred_factors = pd.merge(df_seal,df_whale, on = 'Year',how = 'outer')
df_pred_factors.rename(columns = {'sogNt':'Harbour seal','NRKW':'Killer whale'},inplace = True)
df_input_variables = pd.merge(df_env_factors,df_pred_factors,on = 'Year', how = 'outer')
df_input_variables

,Year,Avg Sea Salinity,pdo,Max flow rate,Min flow rate,Avg flow rate,Harbour seal,Killer whale
0,1996,26.720833,0.683333,8100.0,675.0,4387.5,39166,214.0
1,1997,25.359167,1.316667,11300.0,788.0,6044.0,39187,218.0
2,1998,27.243333,-0.478333,6710.0,600.0,3655.0,39190,216.0
3,1999,26.195833,-1.843333,11000.0,647.0,5823.5,39190,216.0
4,2000,27.405833,-1.125833,8000.0,470.0,4235.0,39190,208.0
5,2001,28.385000,-1.134167,7140.0,478.0,3809.0,39190,200.0
6,2002,27.663333,-0.439167,10600.0,491.0,5545.5,39190,203.0
7,2003,27.967273,0.381667,7300.0,472.0,3886.0,39190,204.0
8,2004,27.438333,-0.224167,6650.0,560.0,3605.0,39190,220.0
9,2005,27.265000,-0.188333,7460.0,805.0,4132.5,39190,230.0


# Andrew

# Importing Data

In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Dependent Variables 
### Salmon viability

In [12]:
# import the salmon survival rate data from Perry et al (2021)
Sal_viab = pd.read_excel('Data/ZooPlanktonPerryData.xlsx', sheet_name=5)

Sal_viab = Sal_viab.rename(columns={'Ocean Entry Year':'Year'}).drop(columns=['Year'])

# take logit of Sal_viab values, because these are proportions in (0,1)
# this is done in the Perry et al paper too
from scipy.special import logit, expit

Sal_viab_logit = logit(Sal_viab).interpolate()

## Independent Variables
### BC and WA regional populations and population trends, Port of Vancouver tonnage cargo, NPGO climate index 

In [13]:
# import the population data for BC Regional Districts
BC_pop = pd.read_csv('Data/BCPop1990.csv')#, index_col=0) # only need index_col=0 to make years the indices
BC_pop = BC_pop.rename(columns={'Unnamed: 0':'Year'}).drop_duplicates() # there's a duplicate row at 2011!

# pre-2001 BC population numbers are only for every five years
# make an empty table for these years, to be interpolated
missing_yrs = np.setdiff1d([i for i in range(1990,2001)],[1991,1996])
missing_yrs_vals = np.empty((29,9,))*np.nan
missing_data = np.vstack((missing_yrs,missing_yrs_vals)).transpose()
BC_missing = pd.DataFrame(missing_data)
BC_missing.columns = BC_pop.columns
BC_pop = pd.concat([BC_missing,BC_pop]).sort_values(by='Year')

# fill in the missing years with some linearly interpolated values where possible
# and where this doesn't fill in a value, just repeat the closest value
BC_pop = BC_pop.interpolate().bfill() 
# fix the indices obtained from concatenating
BC_pop = BC_pop.reset_index().drop(columns=['index'])


# import the population data for BC Regional Districts and WA Counties
WA_pop = pd.read_csv('Data/WAPop1990.csv')# index_col=0)
WA_pop = WA_pop.rename(columns={'Unnamed: 0':'Year'})

In [14]:
# BC Group 1 average
BC_1 = BC_pop.iloc[:,[3,7,9,10,12,14,18,20,25,28,29]]
BC_1_avg = pd.DataFrame(BC_1.mean(axis=1), columns=['BC Gp1 Avg'])
# BC Group 2 average
BC_2 = BC_pop.iloc[:,[2,5,15,17,19]]
BC_2_avg = pd.DataFrame(BC_2.mean(axis=1), columns=['BC Gp2 Avg'])
# BC Group 3 average
BC_3 = BC_pop.iloc[:,[6,8,11,13,16,22]]
BC_3_avg = pd.DataFrame(BC_3.mean(axis=1),columns=['BC Gp3 Avg'])
# BC Group 4 average
BC_4 = BC_pop.iloc[:,[1,4,24,27]]
BC_4_avg = pd.DataFrame(BC_4.mean(axis=1),columns=['BC Gp4 Avg'])
# BC Group 5 average
BC_5 = BC_pop.iloc[:,[21,23,26]]
BC_5_avg = pd.DataFrame(BC_5.mean(axis=1),columns=['BC Gp5 Avg'])

# WA Group 1 average
WA_1 = WA_pop.iloc[:,[1,3,5,6,8,9,13,16,19,20,21,23,27,29,31,32,34,36,37,39]]
WA_1_avg = pd.DataFrame(WA_1.mean(axis=1),columns=['WA Gp1 Avg'])
# WA Group 2 average
WA_2 = WA_pop.iloc[:,[2,4,18,22,24,25,26,28,30,33,35]]
WA_2_avg = pd.DataFrame(WA_2.mean(axis=1),columns=['WA Gp2 Avg'])
# WA Group 3 average
WA_3 = WA_pop.iloc[:,[11,14,15,17,38]]
WA_3_avg = pd.DataFrame(WA_3.mean(axis=1),columns=['WA Gp3 Avg'])
# WA Group 4 average
WA_4 = WA_pop.iloc[:,[7,10,12]]
WA_4_avg = pd.DataFrame(WA_4.mean(axis=1),columns=['WA Gp4 Avg'])

In [15]:
# data on total tonnage of cargo through Port of Vancouver
# years 2008-2023 are given in the Port Metro Vancouver Statistics Overviews at https://www.portvancouver.com/about-us/statistics/
# years 1994-1998 are given in The Institutional Position of Seaports An International Comparison by Henrik Stevens, Chapter 7
PortofVan = pd.DataFrame({'Year':[1994,1995,1996,1997,1998]+[i for i in range(2008,2023)],  
                          'Tonnage':[67600000,71500000,72000000,73500000,71900000]+[114561990,101887824,118378885, 
                                122499631,123876885,135009878, 
                                139638157,138082585,135538055, 
                                142067550,147090934,144225630, 
                                145450722,146473626,141416326]})

missing_yrs_port = pd.DataFrame({'Year':[1990,1991,1992,1993,1999,2000,2001,2002,2003,2004,2005,2006,2007],
                               'Tonnage':[np.nan for _ in range(0,13)]})

# linearly interpolate the port traffic data
PortofVanExt = pd.concat([PortofVan,missing_yrs_port]).sort_values(by=['Year']).interpolate(method='linear', fill_value='extrapolate', limit_direction='both')
PortofVanExt = PortofVanExt.reset_index(drop=True)

In [16]:
# import a climatological index
NPGO_monthly = pd.read_csv('Data/NPGOMonthlyAvg copy.txt')

# get the yearly NPGO averages and clean it up
NPGO_yearly = NPGO_monthly.groupby('Year').mean()
NPGO_yearly = NPGO_yearly.drop(columns=['Month']).reset_index()
NPGO = NPGO_yearly.loc[(NPGO_yearly['Year']>=1990) & (NPGO_yearly['Year']<2023)]
NPGO = NPGO.reset_index().drop(columns=['index'])

In [17]:
BC_df = pd.concat([  BC_1_avg,
                             BC_2_avg,
                             BC_3_avg,
                             BC_4_avg,
                             BC_5_avg ]
                  , axis=1)
                   
WA_df = pd.concat([ WA_1_avg,
                             WA_2_avg,
                             WA_3_avg ,
                             WA_4_avg ]
                  , axis=1)

populations = pd.concat([WA_pop['Year'] , BC_df, WA_df ]  , axis=1) 
                             

In [18]:
display(populations.head(3))
display(NPGO.head(3))
display(PortofVanExt.head(3))

,Year,BC Gp1 Avg,BC Gp2 Avg,BC Gp3 Avg,BC Gp4 Avg,BC Gp5 Avg,WA Gp1 Avg,WA Gp2 Avg,WA Gp3 Avg,WA Gp4 Avg
0,1990,239210.686699,25752.799952,57641.771741,35859.653528,21056.308168,138198.85,34745.090909,341584.6,4189.000000
1,1991,239210.686699,25752.799952,57641.771741,35859.653528,21056.308168,142042.85,35751.727273,350711.6,4223.000000
2,1992,246979.926139,25950.319928,58710.597327,36439.583717,21182.764974,144950.55,36598.181818,355353.8,4259.333333


,Year,NPGO index
0,1990.0,-0.201546
1,1991.0,-0.498915
2,1992.0,-1.094919


,Year,Tonnage
0,1990,67600000.0
1,1991,67600000.0
2,1992,67600000.0


# Aycin

In [19]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("whitegrid")

Years

In [20]:
#Years we concerned with 
start_year=1990
end_year=2017
year_range = range(start_year, end_year+1) 

 Target Data - Salmon Survivals

In [21]:
#Read ZooPlanktonPerryData file
##obtain SalmonSurvivals_data
            #Take years less than or equal to 2017
                            #last row (year 2018) has only NaN


file_path = "Data/ZooPlanktonPerryData.xlsx"
selected_sheet = "5. Salmon marine survivals"  
df = pd.read_excel(file_path, sheet_name=selected_sheet)
SalmonSurvivals_data = df[df["Ocean Entry Year"]<=2017]


#Rename the columns 
abbreviation_mapping = {
    'Ocean Entry Year': 'year',
    'Cowichan Chinook': 'Cow_Ch',
    'Harrison Chinook': 'Har_Ch',
    'Puntledge Chinook': 'Pun_Ch',
    'Big Qualicum Coho': 'BQ_Coho'
}

salmonSurvivals = SalmonSurvivals_data.rename(columns=abbreviation_mapping)


# List of salmon types 
year_column = 'year'
salmonTypes_List =  [col for col in salmonSurvivals.columns if col != year_column]

Data Reads

Total Zooplankton Biomasses

In [22]:
#Read ZooPlanktonPerryData file

##obtain  plankBiomass_data   
###take the years less than 2017 
###we drop columns that do not give biomass of zooplanktons
file_path = "Data/ZooPlanktonPerryData.xlsx"
selected_sheet = "1. Zooplankton Biomass data"  
df = pd.read_excel(file_path, sheet_name=selected_sheet)
df= df[df["yr"]<=2017]
plankBiomass_data=df.drop(columns=['key', 'survey', 'region', 'event', 'net', 'station', 'lon', 'lat', 'mon', 'day', 'time',  'twilight', 'net.type', 'diam.m', 'mesh.um', 'startz.m', 'endz.m','botz.m',
       'volfilt.m3'])

##  BiomassDefs      (if info on column names is needed)
selected_sheet = "2. Biomass data definitions"  
BiomassDefs = pd.read_excel(file_path, sheet_name=selected_sheet)



## Create a new DataFrame which contains years and corresponding total biomasses mean
#Calculate average Total Biomass for each year inside the common year range
#drop duplicates
#reset index

totalBiomass_av= pd.DataFrame()

totalBiomass_av['year']= plankBiomass_data['yr']
totalBiomass_av['Av_totalBiomass']= plankBiomass_data.groupby('yr')['Total.Biomass'].transform('mean')
totalBiomass_av.drop_duplicates(inplace=True)
totalBiomass_av.reset_index(inplace=True, drop=True)


# Create a new DataFrame for averaged values
#Group by 'yr' and calculate the annual mean of Zooplanktons
#drop duplicates
#rename the columns for convenienve
#reset_index

plankBiomassMean = pd.DataFrame()

for column in plankBiomass_data.columns:
    if column in ['yr', 'region']:
        plankBiomassMean[column] = plankBiomass_data[column]
    else:
        name = 'Av_' + column
        plankBiomassMean[name] = plankBiomass_data.groupby('yr')[column].transform('mean')


plankBiomassMean.drop_duplicates(subset=['yr'], inplace=True)
plankBiomassMean.rename(columns={'yr': 'year', 'Av_Total.Biomass': 'Av_totalBiomass'},  inplace=True)
plankBiomassMean.reset_index(drop=True, inplace=True)


### Obtained Data Frames

In [23]:
display(plankBiomassMean.head(3))
display(totalBiomass_av.head(3))
#Note: data of totalBiomass_av appears in plankBiomassMean

,year,Av_AmphiGam,Av_AmphiHyp,Av_BenthicLarv,Av_CalCops.larg,Av_CalCops.med,Av_CalCops.smal,Av_Cephalopoda,Av_Chaetognatha,Av_Cladocera,...,Av_Natantia,Av_NonCalCops,Av_Ostracoda,Av_Other,Av_PolychaetPelagic,Av_Pteropods,Av_Repantia,Av_Scyphozoa,Av_Siphonophorae,Av_totalBiomass
0,1996,3.099832,1.324307,0.274767,13.213023,9.285405,0.929795,0.015525,0.358527,0.000955,...,0.718163,0.322313,2.674950,0.083352,1.615180,0.324095,0.081970,0.0,0.531088,37.388238
1,1997,8.415328,1.458525,0.033581,24.451942,9.381715,0.810660,0.022965,1.561735,0.006467,...,0.606682,0.358207,3.035919,0.041017,0.437920,0.482410,4.063566,0.0,0.431464,71.212723
2,1998,8.007227,3.214592,0.572927,12.494889,11.405672,1.033331,0.053075,0.980033,0.010478,...,0.637259,0.325437,3.960144,0.004294,0.897803,2.674066,0.688282,0.0,0.202136,105.522363


,year,Av_totalBiomass
0,1996,37.388238
1,1997,71.212723
2,1998,105.522363


Sea Level

In [24]:
## Read the Annual Mean Sea Levels
        #there are data of four ports
#Take the years between start_year and end_year
#Take the positive values (non existent data entered as -99999
#drop the columns 2 and 3 to keep year and sea level measurement
#rename the column that contains Sea Level measurement


file_path="Data/SeaLevel_Point_Atkinson193.txt"
df=pd.read_csv(file_path, header=None,delimiter=';')
df= df[(df[0] >= start_year) & (df[0] <= end_year) & (df[1]>=0)].drop(columns=[2,3])
seaLevelPointA_clean =df.rename(columns={0: 'year', 1: 'SeaLevel_Point_Atkinson'})
#missing year 1997

file_path="Data/SeaLevel_Port_Angeles2127.txt"
df =pd.read_csv(file_path, header=None,delimiter=';')
df= df[(df[0] >= start_year) & (df[0] <= end_year) & (df[1]>=0)].drop(columns=[2,3])
seaLevelPortA_clean= df.rename(columns={0: 'year', 1: 'SeaLevel_Port_Angeles'})
#no missing year

file_path="Data/SeaLevel_Campbell_River1323.txt"
df=pd.read_csv(file_path, header=None,delimiter=';')
df= df[(df[0] >= start_year) & (df[0] <= end_year) & (df[1]>=0)].drop(columns=[2,3])
seaLevelCampbellR_clean= df.rename(columns={0: 'year', 1: 'SeaLevel_Campbell_River'})
#missing year 1995 & 1996

file_path="Data/SeaLevel_Cherry_annualMean.txt"
df= pd.read_csv(file_path, header=None,delimiter=';')
df= df[(df[0] >= start_year) & (df[0] <= end_year) & (df[1]>=0)].drop(columns=[2,3])
seaLevelCherryP_clean= df.rename(columns={0: 'year', 1: 'SeaLevel_Cherry'})
#missing year 1994


#Merge the sea Levels
#Calculate their average 
    #missing data is ignored 
        #note: if a year is missing it's only missing at most one of the ports

merged_df= pd.merge(seaLevelPortA_clean, seaLevelPointA_clean, on='year', how='outer')
merged_df = pd.merge(merged_df, seaLevelCampbellR_clean, on='year', how='outer')
seaLevels = pd.merge(merged_df, seaLevelCherryP_clean, on='year', how='outer')

seaLevels['av_SeaLevels']= seaLevels.iloc[:,1:].mean(axis=1, skipna=True)


### Obtained Data Frames

In [25]:
#seaLevels.head(3)

# Observed Features 

In [26]:
AllDataNoNull.columns

Index(['Year', 'Cowichan Chinook', 'Harrison Chinook', 'Puntledge Chinook',
       'Big Qualicum Coho', 'SST Jan.', 'SST Feb.', 'SST Mar.', 'SST Apr.',
       'SST May', 'SST Jun.', 'SST Jul.', 'SST Aug', 'SST Sep.', 'SST Oct.',
       'SST Nov.', 'SST Dec.', 'TotBiom', 'AmphiGam', 'AmphiHyp',
       'BenthicLarv', 'CalCops.larg', 'CalCops.med', 'CalCops.smal',
       'Chaetognatha', 'Cladocera', 'Ctenophora', 'Euphs', 'Fish', 'Larvacea',
       'Medusae', 'Natantia', 'NonCalCops', 'Ostracoda', 'PolychaetPelagic',
       'Pteropods', 'Repantia', 'Siphonophorae', 'Coho', 'Chinook',
       'Sea Level'],
      dtype='object')

In [27]:
df_env_factors.columns

Index(['Year', 'Avg Sea Salinity', 'pdo', 'Max flow rate', 'Min flow rate',
       'Avg flow rate'],
      dtype='object')

In [28]:
df_pred_factors.columns

Index(['Year', 'Harbour seal', 'Killer whale'], dtype='object')

In [29]:
# df_input_variables merges the above two 
                                #df_env_factors and df_pred_factors
# df_input_variables.columns

In [30]:
populations.columns
#merges BC_df, WA_df and 'Year'

Index(['Year', 'BC Gp1 Avg', 'BC Gp2 Avg', 'BC Gp3 Avg', 'BC Gp4 Avg',
       'BC Gp5 Avg', 'WA Gp1 Avg', 'WA Gp2 Avg', 'WA Gp3 Avg', 'WA Gp4 Avg'],
      dtype='object')

In [31]:
NPGO.columns

Index(['Year', 'NPGO index'], dtype='object')

In [32]:
PortofVanExt.columns

Index(['Year', 'Tonnage'], dtype='object')

In [33]:
plankBiomassMean.columns
# totalBiomass_av is 'Av_totalBiomass' the same as plankBiomassMean[['year', 'Av_totalBiomass']]

Index(['year', 'Av_AmphiGam', 'Av_AmphiHyp', 'Av_BenthicLarv',
       'Av_CalCops.larg', 'Av_CalCops.med', 'Av_CalCops.smal',
       'Av_Cephalopoda', 'Av_Chaetognatha', 'Av_Cladocera', 'Av_Ctenophora',
       'Av_Euphs', 'Av_Fish', 'Av_Larvacea', 'Av_Medusae', 'Av_Mysids',
       'Av_Natantia', 'Av_NonCalCops', 'Av_Ostracoda', 'Av_Other',
       'Av_PolychaetPelagic', 'Av_Pteropods', 'Av_Repantia', 'Av_Scyphozoa',
       'Av_Siphonophorae', 'Av_totalBiomass'],
      dtype='object')

In [34]:
seaLevels.columns
###Some columns contain NaN values

Index(['year', 'SeaLevel_Port_Angeles', 'SeaLevel_Point_Atkinson',
       'SeaLevel_Campbell_River', 'SeaLevel_Cherry', 'av_SeaLevels'],
      dtype='object')